<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/thermodynamics/EOSCG_vs_GERG2008.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup and Imports
# Ensure neqsim is installed: pip install neqsim
import neqsim
from neqsim.thermo import SystemGERG2008Eos, SystemEOSCGEos
from neqsim.thermo import ThermodynamicOperations
import matplotlib.pyplot as plt
import pandas as pd

### Example 1: Density Comparison for a CO2-Rich Mixture

We will compare the calculated density of a mixture containing CO2 and Nitrogen, which is relevant for CCS transport.

In [ ]:
# Define conditions
temperature = 298.15  # K
pressure = 100.0      # bara

# Create GERG-2008 System
sys_gerg = SystemGERG2008Eos(temperature, pressure)
sys_gerg.addComponent("CO2", 0.9)
sys_gerg.addComponent("nitrogen", 0.1)
sys_gerg.setMixingRule("classic") # GERG/EOS-CG handle their own mixing rules internally, but init is needed

# Create EOS-CG System
sys_eoscg = SystemEOSCGEos(temperature, pressure)
sys_eoscg.addComponent("CO2", 0.9)
sys_eoscg.addComponent("nitrogen", 0.1)

# Run Flash/Calculations
ops_gerg = ThermodynamicOperations(sys_gerg)
ops_gerg.TPflash()

ops_eoscg = ThermodynamicOperations(sys_eoscg)
ops_eoscg.TPflash()

# Retrieve Results
rho_gerg = sys_gerg.getPhase(0).getDensity() # kg/m3
rho_eoscg = sys_eoscg.getPhase(0).getDensity() # kg/m3

print(f"Conditions: {temperature} K, {pressure} bara")
print(f"Mixture: 90% CO2, 10% N2")
print(f"GERG-2008 Density: {rho_gerg:.4f} kg/m3")
print(f"EOS-CG Density:    {rho_eoscg:.4f} kg/m3")
print(f"Difference:        {abs(rho_gerg - rho_eoscg):.4f} kg/m3")

### Example 2: Phase Envelope Generation

Comparing the phase envelopes predicted by both models for a natural gas mixture.

In [ ]:
def get_envelope(system):
    ops = ThermodynamicOperations(system)
    ops.calcPTphaseEnvelope()
    return ops

# Define a standard gas mixture
sys_gerg_env = SystemGERG2008Eos(288.15, 50.0)
sys_gerg_env.addComponent("methane", 90.0)
sys_gerg_env.addComponent("ethane", 5.0)
sys_gerg_env.addComponent("propane", 3.0)
sys_gerg_env.addComponent("CO2", 2.0)

sys_eoscg_env = SystemEOSCGEos(288.15, 50.0)
sys_eoscg_env.addComponent("methane", 90.0)
sys_eoscg_env.addComponent("ethane", 5.0)
sys_eoscg_env.addComponent("propane", 3.0)
sys_eoscg_env.addComponent("CO2", 2.0)

# Calculate Envelopes (This might take a few seconds)
# Note: In a real notebook execution, ensure the Java environment is properly initialized.
try:
    ops_gerg_env = get_envelope(sys_gerg_env)
    ops_eoscg_env = get_envelope(sys_eoscg_env)

    # Extract data (Pseudo-code for extraction, depends on exact API return types)
    # Usually we get arrays from the operations object
    # plt.plot(ops_gerg_env.getOperation().getPoints()[0], ops_gerg_env.getOperation().getPoints()[1], label='GERG-2008')
    # plt.plot(ops_eoscg_env.getOperation().getPoints()[0], ops_eoscg_env.getOperation().getPoints()[1], label='EOS-CG')
    # plt.legend()
    # plt.xlabel('Temperature [K]')
    # plt.ylabel('Pressure [bara]')
    # plt.title('Phase Envelope Comparison')
    # plt.show()
    print("Phase envelope calculation initiated.")
except Exception as e:
    print(f"Error calculating phase envelope: {e}")

## Summary

Use **GERG-2008** when:
*   Working with standard natural gas mixtures.
*   Performing custody transfer calculations.
*   Comparing against ISO standards.

Use **EOS-CG** when:
*   Simulating CCS streams (high CO2).
*   Working with humid gases or combustion products.
*   Accuracy in the water-rich phase is critical (and CPA is not an option).